In [ ]:
import torch
torch.cuda.set_device(3)
print(torch.cuda.current_device())

In [ ]:
%run prepare_data.py
%run architectures.py

## MobileNet

#### RSNA

In [3]:
train_set, test_set = prepare_rsna_datasamples()

In [4]:
valid_ds = rsna_dataset(test_set, transform=False)
valid_dataloader = DataLoader(valid_ds, batch_size=32, num_workers=4)

In [5]:
p = Path("Scaling_experiments/RSNA/")
ls = list(p.iterdir())
models = [str(s) for s in ls if '_' in str(s) and 'ipynb' not in str(s) and '.csv' not in str(s) 
          and 'Mobile' in str(s)]

In [ ]:
data = []

for model_p in models:
    w = float(model_p.split('_')[-2])
    d = float(model_p.split('_')[-1])
    model = MobileNet(num_classes=1, width_mult=w, depth_mult=d).cuda()
    load_model(model, model_p)
    for i in range(15):
        test_sample = test_set.sample(frac=1.0, replace=True)
        valid_ds = rsna_dataset(test_sample, transform=False)
        valid_dl = DataLoader(valid_ds, batch_size=32, num_workers=4)
        _, score = val_metrics_rsna(model, valid_dl)
        data.append([model_p, score])
#         print(val_metrics_rsna(model, valid_dl))

In [ ]:
rsna_mobile = pd.DataFrame(data=data, columns=['model', 'score'])

In [ ]:
rsna_mobile.to_csv('bootstrap/rsna_mobile.csv', index=False)

#### Chexpert

In [ ]:
PATH = Path('/mnt/wamri/gilmer/chexpert/ChesXPert-250')
train_df = pd.read_csv('/home/rimmanni/experiments/chexpert/train_df_chexpert_resized.csv')
valid_df = pd.read_csv('/home/rimmanni/experiments/chexpert/valid_df_chexpert_resized.csv')

In [ ]:
p = Path("Scaling_experiments/Chexpert/")
ls = list(p.iterdir())
models = [str(s) for s in ls if '_' in str(s) and 'ipynb' not in str(s) and '.csv' not in str(s) 
          and 'Mobile' in str(s)]

In [ ]:
data = []

for model_p in models:
    w = float(model_p.split('_')[-2])
    d = float(model_p.split('_')[-1])
    model = MobileNet(num_classes=5, width_mult=w, depth_mult=d).cuda()
    load_model(model, model_p)
    for i in range(15):
        test_sample = valid_df.sample(frac=1.0, replace=True)
        valid_ds = chexpert_dataset(test_sample, image_path=PATH, transform=False)
        valid_loader = DataLoader(valid_ds, batch_size=32, num_workers=4)
        _, score = val_metrics_chexpert(model, valid_loader)
        data.append([model_p, score])
#         print(val_metrics_chexpert(model, valid_loader))

In [ ]:
chexpert_mobile = pd.DataFrame(data=data, columns=['model', 'score'])

In [ ]:
chexpert_mobile.to_csv('bootstrap/chexpert_mobile.csv', index=False)

## Resnet_depth

In [3]:
depths = [[[[64, 2], [128, 2]], [[256, 2], [512, 1]]],
          [[[64, 2], [128, 2]], [[256, 1], [512, 1]]],
          [[[64, 2], [128, 1]], [[256, 1], [512, 1]]],
          [[[64, 2], [128, 1]], [[256, 2], [512, 1]]],
          [[[64, 1], [128, 1]], [[256, 2], [512, 1]]],
          [[[64, 1], [128, 1]], [[256, 1], [512, 1]]],
         ]

#### RSNA

In [4]:
train_set, test_set = prepare_rsna_datasamples()

In [5]:
p = Path("Scaling_experiments/RSNA/")
ls = list(p.iterdir())
models = [str(s) for s in ls if 'ResDepth' in str(s)]

In [6]:
data = []

for model_p in models:
    w = float(model_p.split('_')[-2])
    i = int(model_p.split('_')[-1])
    d = depths[i]
    model = resnet18(block=depthwise_block, width_mult=w, 
                     inverted_residual_setting1=d[0], 
                     inverted_residual_setting2=d[1]).cuda()

    load_model(model, model_p)
    for i in range(15):
        test_sample = test_set.sample(frac=1.0, replace=True)
        valid_ds = rsna_dataset(test_sample, transform=False)
        valid_dl = DataLoader(valid_ds, batch_size=32, num_workers=4)
        _, score = val_metrics_rsna(model, valid_dl)
        data.append([model_p, score])
#         print(val_metrics_rsna(model, valid_dl))

In [7]:
rsna_rd = pd.DataFrame(data=data, columns=['model', 'score'])

In [8]:
rsna_rd.to_csv('bootstrap/rsna_resdepth.csv', index=False)

#### Chexpert

In [9]:
PATH = Path('/mnt/wamri/gilmer/chexpert/ChesXPert-250')
train_df = pd.read_csv('/home/rimmanni/experiments/chexpert/train_df_chexpert_resized.csv')
valid_df = pd.read_csv('/home/rimmanni/experiments/chexpert/valid_df_chexpert_resized.csv')

In [10]:
p = Path("Scaling_experiments/Chexpert/")
ls = list(p.iterdir())
models = [str(s) for s in ls if 'ResDepth' in str(s)]

In [11]:
data = []

for model_p in models:
    w = float(model_p.split('_')[-2])
    i = int(model_p.split('_')[-1])
    d = depths[i]
    model = resnet18(num_classes=5, block=depthwise_block, width_mult=w, 
                     inverted_residual_setting1=d[0], 
                     inverted_residual_setting2=d[1]).cuda()

    load_model(model, model_p)
    for i in range(15):
        test_sample = valid_df.sample(frac=1.0, replace=True)
        valid_ds = chexpert_dataset(test_sample, image_path=PATH, transform=False)
        valid_loader = DataLoader(valid_ds, batch_size=32, num_workers=4)
        _, score = val_metrics_chexpert(model, valid_loader)
        data.append([model_p, score])
#         print(val_metrics_chexpert(model, valid_loader))

In [12]:
chexpert_rd = pd.DataFrame(data=data, columns=['model', 'score'])

In [13]:
chexpert_rd.to_csv('bootstrap/chexpert_resdepth.csv', index=False)

In [5]:
p = Path("Scaling_experiments/Mura/")
ls = list(p.iterdir())
models = [str(s) for s in ls if '_' in str(s) and 'ipynb' not in str(s) and '.csv' not in str(s) 
          and 'Mobile' in str(s)]

valid_all = set('_'.join(f.split('_')[3:7]) for f in mura_df.path)

In [6]:
for model_p in models[0:1]:
    w = float(model_p.split('_')[-2])
    d = float(model_p.split('_')[-1])
    model = MobileNet(num_classes=1, width_mult=w, depth_mult=d).cuda()
    load_model(model, model_p)
    for i in range(10):
        sampled = resample(list(valid_all), replace=True)
        mura_df_sampled = mura_df.loc[(mura_df.type.isin(sampled))]
        valid_ds = mura_ds(mura_df_sampled)
        valid_dl = DataLoader(valid_ds, batch_size=64,num_workers=4)
        print(val_metrics_mura(model, valid_dl, valid_ds))

(1.0753825690665497, 0.538454405386898)
(1.0733709574461503, 0.5416139348166292)
(1.1127038200817634, 0.5318014804484257)
(1.067314435010753, 0.5363489043894878)
(1.0790780696410078, 0.5420631107988155)
(1.0633329537029519, 0.5255430873255647)
(1.0748467968880606, 0.5304794243673701)
(1.1015908149785774, 0.5495230945862404)
(1.1043104505875558, 0.520367990535055)
(1.0644899012962183, 0.5340070464398081)


In [10]:
model_p = models[0]
w = float(model_p.split('_')[-2])
d = float(model_p.split('_')[-1])
w, d

model = MobileNet(num_classes=1, width_mult=w, depth_mult=d).cuda()
load_model(model, model_p)

In [8]:
sampled = resample(list(valid_all), replace=True)
paths = [p for p in mura_df.path]
mura_df_sampled = mura_df.loc[(mura_df.path.isin(sampled))]
valid_ds = mura_ds(mura_df)
valid_dl = DataLoader(valid_ds, batch_size=64,num_workers=4)

In [10]:
val_metrics_mura(model, valid_dl, valid_ds)

(1.0843127500528016, 0.530147517842179)

In [19]:
mura_df['type'] = mura_df.path.apply(lambda x: '_'.join(x.split('_')[3:7]))

In [20]:
mura_df

,path,label,type
0,/home/rimmanni/data/mura/valid_250_200/1/XR_SH...,1,SHOULDER_patient11725_study1_positive
1,/home/rimmanni/data/mura/valid_250_200/1/XR_FO...,1,FOREARM_patient11428_study1_positive
2,/home/rimmanni/data/mura/valid_250_200/1/XR_FI...,1,FINGER_patient11893_study1_positive
3,/home/rimmanni/data/mura/valid_250_200/1/XR_WR...,1,WRIST_patient11194_study1_positive
4,/home/rimmanni/data/mura/valid_250_200/1/XR_FI...,1,FINGER_patient11410_study1_positive
5,/home/rimmanni/data/mura/valid_250_200/1/XR_EL...,1,ELBOW_patient11358_study1_positive
6,/home/rimmanni/data/mura/valid_250_200/1/XR_HA...,1,HAND_patient11516_study1_positive
7,/home/rimmanni/data/mura/valid_250_200/1/XR_HA...,1,HAND_patient11497_study1_positive
8,/home/rimmanni/data/mura/valid_250_200/1/XR_FI...,1,FINGER_patient11921_study1_positive
9,/home/rimmanni/data/mura/valid_250_200/1/XR_FI...,1,FINGER_patient11578_study1_positive
